# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide. 

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [21]:
# your code here
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ssl
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')




[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [22]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/your-code/training.1600000.processed.noemoticon.csv', encoding='latin-1')
data.columns = ['target','id','date','flag','user','text']
data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [26]:
data = data.sample(20000)
data.head()

,target,id,date,flag,user,text,text_processed
1217649,4,1989756606,Mon Jun 01 02:42:00 PDT 2009,NO_QUERY,Young_SoulBlade,"@GinuwinelyLyric CiCi-Boards A Shoutout, Deadd...","[ginuwinelylyric, cici-boards, a, shoutout, ,,..."
1384950,4,2052703377,Sat Jun 06 01:41:45 PDT 2009,NO_QUERY,thisisjoo,i am lovin' WOWP''s episode today.,"[i, be, lovin, wowp, s, episode, today, .]"
28940,0,1559896382,Sun Apr 19 12:19:31 PDT 2009,NO_QUERY,Anyelday,@heatherbookman haha My mom never had that stu...,"[heatherbookman, haha, my, mom, never, have, t..."
1581250,4,2190162694,Tue Jun 16 01:56:58 PDT 2009,NO_QUERY,Kauritree,@stephenfry Your twitter hating friend Hugh La...,"[stephenfry, your, twitter, hat, friend, hugh,..."
190106,0,1969283383,Fri May 29 23:33:26 PDT 2009,NO_QUERY,lalayu,@ItsJustDi yes...but she didnt want to sit in ...,"[itsjustdi, yes, ..., but, she, didnt, want, t..."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [27]:
# your code here
patterns= ["http(s)?:\/\/[\w]{3,}.[\w]{2,}","\[\([\d]{2,}\)\]\"\)","@","-#", "[\d]{2,}-", "'"]


def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    for i in range(len(patterns)):
        if i == 0:
            new=s
        else:
            new=new
        new= re.sub(patterns[i], ' ', new)
    return new.lower()

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    
    tokens = word_tokenize(s)
    return tokens

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper() 
        tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
        return tag_dict.get(tag, wordnet.NOUN)

    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in l]
    return lemmatized

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    no_sw = [word for word in l if not word in stopwords.words()]
    return(no_sw)



data['text_processed'] = data['text'].apply(clean_up)
data['text_processed'] = data['text_processed'].apply(tokenize)
data['text_processed'] = data['text_processed'].apply(stem_and_lemmatize)
data['text_processed'] = data['text_processed'].apply(remove_stopwords)


data.head()

,target,id,date,flag,user,text,text_processed
1217649,4,1989756606,Mon Jun 01 02:42:00 PDT 2009,NO_QUERY,Young_SoulBlade,"@GinuwinelyLyric CiCi-Boards A Shoutout, Deadd...","[ginuwinelylyric, cici-boards, shoutout, ,, de..."
1384950,4,2052703377,Sat Jun 06 01:41:45 PDT 2009,NO_QUERY,thisisjoo,i am lovin' WOWP''s episode today.,"[lovin, wowp, episode, today, .]"
28940,0,1559896382,Sun Apr 19 12:19:31 PDT 2009,NO_QUERY,Anyelday,@heatherbookman haha My mom never had that stu...,"[heatherbookman, haha, mom, stuff, ., hfc, .]"
1581250,4,2190162694,Tue Jun 16 01:56:58 PDT 2009,NO_QUERY,Kauritree,@stephenfry Your twitter hating friend Hugh La...,"[stephenfry, twitter, friend, hugh, laurie, re..."
190106,0,1969283383,Fri May 29 23:33:26 PDT 2009,NO_QUERY,lalayu,@ItsJustDi yes...but she didnt want to sit in ...,"[itsjustdi, ..., ..., ...]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [28]:
# your code here
data['text_processed'][0:10]


1217649    [ginuwinelylyric, cici-boards, shoutout, ,, de...
1384950                     [lovin, wowp, episode, today, .]
28940          [heatherbookman, haha, mom, stuff, ., hfc, .]
1581250    [stephenfry, twitter, friend, hugh, laurie, re...
190106                            [itsjustdi, ..., ..., ...]
894970                      [john_af, hey, john, !, wave, *]
438212     [jensenclan88, omg, !, !, starbucks, twice-, m...
562157         [baby, oil, ., dress, ., hair, ., miss, ....]
69395                                     [., wait, week, .]
62682                   [starbucks, .., english, paper, day]
Name: text_processed, dtype: object

In [31]:
word_list = []

for l in data['text_processed']:
    word_list += l

#word_list



In [33]:
from nltk.probability import FreqDist

fd = FreqDist()
for word in word_list:
    fd[word.lower()] += 1

fd

FreqDist({'!': 11308, '.': 9977, ',': 6098, '?': 3364, '...': 2693, ';': 2178, '&': 1943, '..': 1351, 'day': 1290, 'work': 1102, ...})

In [42]:
most_freq=fd.most_common(5000)

most_freq

[('!', 11308),
 ('.', 9977),
 (',', 6098),
 ('?', 3364),
 ('...', 2693),
 (';', 2178),
 ('&', 1943),
 ('..', 1351),
 ('day', 1290),
 ('work', 1102),
 ('love', 961),
 ('quot', 930),
 ('-', 927),
 ('time', 858),
 ('today', 803),
 (':', 776),
 ('back', 746),
 ('miss', 702),
 ('make', 696),
 ('amp', 668),
 ('feel', 633),
 (')', 563),
 ('*', 562),
 ('night', 560),
 ('watch', 544),
 ('#', 542),
 ('(', 537),
 ('2', 483),
 ('morning', 482),
 ('home', 479),
 ('hope', 472),
 ('twitter', 456),
 ('tomorrow', 441),
 ('wish', 418),
 ('....', 415),
 ('sleep', 409),
 ('haha', 407),
 ('great', 404),
 ('wait', 392),
 ('sad', 370),
 ('3', 365),
 ('week', 358),
 ('fun', 355),
 ('friend', 330),
 ('tonight', 324),
 ('happy', 319),
 ('gon', 317),
 ('sorry', 313),
 ('start', 291),
 ('hour', 287),
 ('nice', 279),
 ('bed', 271),
 ('hate', 268),
 ('tweet', 261),
 ('show', 261),
 ('lt', 256),
 ('school', 255),
 ('follow', 252),
 ('guy', 244),
 ('weekend', 235),
 ('hey', 232),
 ('long', 225),
 ('life', 224),
 ('gi

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [43]:
# your code here
cols = {word[0]: [] for word in most_freq}
words = pd.DataFrame(cols)

words

,!,.,",",?,...,;,&,..,day,work,...,scramble,pussycat,carole29,dougie,8-,liverpool,doze,zzzz,enuff,acc


In [46]:
def find_features(document):
  words=list(document)
  features={}
  most_frequent_words = [ tupple[0] for tupple in most_freq ]
  for word in most_frequent_words:
    features[word] = (word in words)
   
  return features

In [47]:
data['features'] = data['text_processed'].apply(find_features)
data.head()

,target,id,date,flag,user,text,text_processed,features
1217649,4,1989756606,Mon Jun 01 02:42:00 PDT 2009,NO_QUERY,Young_SoulBlade,"@GinuwinelyLyric CiCi-Boards A Shoutout, Deadd...","[ginuwinelylyric, cici-boards, shoutout, ,, de...","{'!': True, '.': False, ',': True, '?': False,..."
1384950,4,2052703377,Sat Jun 06 01:41:45 PDT 2009,NO_QUERY,thisisjoo,i am lovin' WOWP''s episode today.,"[lovin, wowp, episode, today, .]","{'!': False, '.': True, ',': False, '?': False..."
28940,0,1559896382,Sun Apr 19 12:19:31 PDT 2009,NO_QUERY,Anyelday,@heatherbookman haha My mom never had that stu...,"[heatherbookman, haha, mom, stuff, ., hfc, .]","{'!': False, '.': True, ',': False, '?': False..."
1581250,4,2190162694,Tue Jun 16 01:56:58 PDT 2009,NO_QUERY,Kauritree,@stephenfry Your twitter hating friend Hugh La...,"[stephenfry, twitter, friend, hugh, laurie, re...","{'!': False, '.': True, ',': False, '?': False..."
190106,0,1969283383,Fri May 29 23:33:26 PDT 2009,NO_QUERY,lalayu,@ItsJustDi yes...but she didnt want to sit in ...,"[itsjustdi, ..., ..., ...]","{'!': False, '.': False, ',': False, '?': Fals..."


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [48]:
# your code here
words=words.reset_index()


In [50]:
target=data[['target']]
target

,target
1217649,4
1384950,4
28940,0
1581250,4
190106,0
...,...
1243860,4
1173556,4
395458,0
390024,0


In [51]:
target=target[['target']].replace(0,False,regex=True)
target=target[['target']].replace(4,True,regex=True)
target

,target
1217649,True
1384950,True
28940,False
1581250,True
190106,False
...,...
1243860,True
1173556,True
395458,False
390024,False


In [62]:
target = target.rename(columns={'target': 'is_positive'})
target=target.reset_index()


In [63]:
target = target.drop(columns=['index'])

target

,is_positive
0,True
1,True
2,False
3,True
4,False
...,...
19995,True
19996,True
19997,False
19998,False


In [64]:
new=pd.concat([words,target],axis=1)
new

,level_0,!,.,",",?,...,;,&,..,day,...,pussycat,carole29,dougie,8-,liverpool,doze,zzzz,enuff,acc,is_positive
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
19996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
19997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
19998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [66]:
data.reset_index(drop=True,inplace=True)
data.head()

,target,id,date,flag,user,text,text_processed,features
0,4,1989756606,Mon Jun 01 02:42:00 PDT 2009,NO_QUERY,Young_SoulBlade,"@GinuwinelyLyric CiCi-Boards A Shoutout, Deadd...","[ginuwinelylyric, cici-boards, shoutout, ,, de...","{'!': True, '.': False, ',': True, '?': False,..."
1,4,2052703377,Sat Jun 06 01:41:45 PDT 2009,NO_QUERY,thisisjoo,i am lovin' WOWP''s episode today.,"[lovin, wowp, episode, today, .]","{'!': False, '.': True, ',': False, '?': False..."
2,0,1559896382,Sun Apr 19 12:19:31 PDT 2009,NO_QUERY,Anyelday,@heatherbookman haha My mom never had that stu...,"[heatherbookman, haha, mom, stuff, ., hfc, .]","{'!': False, '.': True, ',': False, '?': False..."
3,4,2190162694,Tue Jun 16 01:56:58 PDT 2009,NO_QUERY,Kauritree,@stephenfry Your twitter hating friend Hugh La...,"[stephenfry, twitter, friend, hugh, laurie, re...","{'!': False, '.': True, ',': False, '?': False..."
4,0,1969283383,Fri May 29 23:33:26 PDT 2009,NO_QUERY,lalayu,@ItsJustDi yes...but she didnt want to sit in ...,"[itsjustdi, ..., ..., ...]","{'!': False, '.': False, ',': False, '?': Fals..."


In [68]:
a_list = []
for i in range(10):
  a_list += [(item[0],item[1]) for item in zip(list(data.iloc[i*2000:(i+1)*2000,-1]),list(new.iloc[i*2000:(i+1)*2000,-1]))]
  
a_list[0]

({'!': True,
  '.': False,
  ',': True,
  '?': False,
  '...': False,
  ';': False,
  '&': False,
  '..': False,
  'day': False,
  'work': False,
  'love': False,
  'quot': False,
  '-': False,
  'time': False,
  'today': False,
  ':': False,
  'back': False,
  'miss': False,
  'make': False,
  'amp': False,
  'feel': False,
  ')': False,
  '*': False,
  'night': False,
  'watch': False,
  '#': False,
  '(': False,
  '2': False,
  'morning': False,
  'home': False,
  'hope': False,
  'twitter': False,
  'tomorrow': False,
  'wish': False,
  '....': False,
  'sleep': False,
  'haha': False,
  'great': False,
  'wait': False,
  'sad': False,
  '3': False,
  'week': False,
  'fun': False,
  'friend': False,
  'tonight': False,
  'happy': False,
  'gon': False,
  'sorry': False,
  'start': False,
  'hour': False,
  'nice': False,
  'bed': False,
  'hate': False,
  'tweet': False,
  'show': False,
  'lt': False,
  'school': False,
  'follow': False,
  'guy': False,
  'weekend': False,
  'he

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [69]:
# your code here
training_set=a_list[:1000]
testing_set=a_list[1000:2000]

In [70]:
classifier=nltk.NaiveBayesClassifier.train(training_set)


In [71]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 63.7


In [72]:
classifier.show_most_informative_features(15)


Most Informative Features
                     hey = True             True : False  =      8.8 : 1.0
                  follow = True             True : False  =      8.1 : 1.0
                    hate = True            False : True   =      7.3 : 1.0
                   leave = True            False : True   =      6.6 : 1.0
                     eat = True             True : False  =      6.0 : 1.0
                    baby = True            False : True   =      4.7 : 1.0
                    suck = True            False : True   =      4.7 : 1.0
                    turn = True            False : True   =      4.7 : 1.0
                  excite = True             True : False  =      4.6 : 1.0
                    song = True             True : False  =      4.6 : 1.0
                    busy = True             True : False  =      3.9 : 1.0
                    quot = True             True : False  =      3.7 : 1.0
                      10 = True            False : True   =      3.5 : 1.0

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here